In [3]:
#!pip install pymongo

In [5]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [6]:
client = MongoClient('localhost', 27017)

In [7]:
db = client.test_database

In [8]:
db = client['test-database']

In [9]:
collection = db.test_collection

In [10]:
collection = db['test-collection']

In [11]:
import datetime
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [12]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5e88525943b039f384de5fea')

In [13]:
db.list_collection_names()

['posts']

In [14]:
import pprint

In [15]:
pprint.pprint(posts.find_one())

{'_id': ObjectId('5e88525943b039f384de5fea'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 4, 4, 9, 23, 55, 577000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [16]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('5e88525943b039f384de5fea'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 4, 4, 9, 23, 55, 577000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [17]:
posts.find_one({"author": "Eliot"})

In [18]:
post_id

ObjectId('5e88525943b039f384de5fea')

In [19]:
post_id_as_str = str(post_id)

In [20]:
posts.find_one({"_id": post_id_as_str}) # No result

In [21]:
from bson.objectid import ObjectId

In [22]:
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

In [23]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [24]:
result = posts.insert_many(new_posts)

In [25]:
result.inserted_ids

[ObjectId('5e88532b43b039f384de5feb'), ObjectId('5e88532b43b039f384de5fec')]

In [26]:
for post in posts.find():
...   pprint.pprint(post)

{'_id': ObjectId('5e88525943b039f384de5fea'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 4, 4, 9, 23, 55, 577000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5e88532b43b039f384de5feb'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5e88532b43b039f384de5fec'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


In [27]:
for post in posts.find({"author": "Mike"}):
...   pprint.pprint(post)

{'_id': ObjectId('5e88525943b039f384de5fea'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 4, 4, 9, 23, 55, 577000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5e88532b43b039f384de5feb'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


In [28]:
posts.count_documents({})

3

In [29]:
posts.count_documents({"author": "Mike"})

2

In [30]:
d = datetime.datetime(2009, 11, 12, 12)

In [31]:
for post in posts.find({"date": {"$lt": d}}).sort("author"):
...   pprint.pprint(post)

{'_id': ObjectId('5e88532b43b039f384de5fec'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5e88532b43b039f384de5feb'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


In [32]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)],
...                                   unique=True)

In [33]:
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

In [34]:
user_profiles = [
...     {'user_id': 211, 'name': 'Luke'},
...     {'user_id': 212, 'name': 'Ziltoid'}]

In [35]:
result = db.profiles.insert_many(user_profiles)

In [37]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile)  # This is fine.
result = db.profiles.insert_one(duplicate_profile)

DuplicateKeyError: E11000 duplicate key error collection: test-database.profiles index: user_id_1 dup key: { : 212 }